## **Setting up the Development Environment**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd /content/drive/MyDrive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/Task 2

/content/drive/MyDrive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/Task 2


In [4]:
! pip install -U -q transformers
! pip install -U -q sentencepiece

     |████████████████████████████████| 2.9 MB 5.1 MB/s 
     |████████████████████████████████| 56 kB 4.6 MB/s 
     |████████████████████████████████| 3.3 MB 46.6 MB/s 
     |████████████████████████████████| 895 kB 50.3 MB/s 
     |████████████████████████████████| 636 kB 57.8 MB/s 
     |████████████████████████████████| 1.2 MB 4.9 MB/s 


## **Importing Libraries**

In [5]:
import pandas as pd

import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from transformers import AutoTokenizer, AutoModel

import pickle

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## **Loading Data**

In [6]:
dataframe = pd.read_csv("Dataset/hasoc_tamil_task2_withlabels.tsv", sep='\t')

In [7]:
dataframe.head()

,id,text,category
0,Tam_1,Indha movie ku award tharlana avanga mansanay ...,OFF
1,Tam_2,kritheeck Kookaburra en unaku enachu? Cbsc ah??,NOT
2,Tam_3,Actually Oru particular bus incident thalaiya ...,OFF
3,Tam_4,Small suggestions: mic ah shirt la pottukunga bro,NOT
4,Tam_5,Karnan padathulaa oru pombaa varumlaa athuu en...,NOT


In [8]:
dataframe.describe()

,id,text,category
count,1001,1001,1000
unique,1001,994,2
top,Tam_614,Indha movie ku award tharlana avanga mansanay ...,NOT
freq,1,3,605


In [9]:
dataframe['category'].unique()

array(['OFF', 'NOT', nan], dtype=object)

In [10]:
dataframe.shape

(1001, 3)

In [11]:
dataframe = dataframe.dropna()
dataframe['category'].unique()

array(['OFF', 'NOT'], dtype=object)

In [12]:
dataframe.shape

(1000, 3)

In [13]:
text = dataframe['text']
text

0       Indha movie ku award tharlana avanga mansanay ...
1         kritheeck Kookaburra en unaku enachu? Cbsc ah??
2       Actually Oru particular bus incident thalaiya ...
3       Small suggestions: mic ah shirt la pottukunga bro
4       Karnan padathulaa oru pombaa varumlaa athuu en...
                              ...                        
996     Chai spoiler vaya mooduya full kathayum sillit...
997     Unakku thinga soru irukko illayo aduthavangala...
998     Dai seripu oli advingi mala vandavan thana mau...
999     Gomala bule shirt ta nee padam eduthu paaruda ...
1000    Mr. Maran neenga mindvoice nu nenachu sound ah...
Name: text, Length: 1000, dtype: object

In [14]:
label = dataframe['category']
label

0       OFF
1       NOT
2       OFF
3       NOT
4       NOT
       ... 
996     OFF
997     OFF
998     OFF
999     OFF
1000    NOT
Name: category, Length: 1000, dtype: object

## **Data Preprocessing**

In [15]:
le = LabelEncoder()
label = le.fit_transform(label)
# label

In [16]:
text = text.str.replace(r"[+/#@&*$%:]",'')
text = text.to_numpy()
# text

In [17]:
english_stopwords = stopwords.words("english")
english_stemmer = SnowballStemmer("english")

def clean_text(text):
    tokens = []
    for token in text.split():
        if token not in english_stopwords:
            token = english_stemmer.stem(token)
            tokens.append(token)
    return " ".join(tokens)

text = [clean_text(i) for i in text]

text = np.array(text)

## **Feature Extraction**

In [18]:
# Load Transformer Model

tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
model = AutoModel.from_pretrained("ai4bharat/indic-bert")

Downloading:   0%|          | 0.00/507 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.38M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/129M [00:00<?, ?B/s]

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertModel: ['predictions.decoder.weight', 'sop_classifier.classifier.weight', 'predictions.LayerNorm.weight', 'predictions.dense.bias', 'predictions.dense.weight', 'predictions.bias', 'predictions.LayerNorm.bias', 'predictions.decoder.bias', 'sop_classifier.classifier.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [19]:
# Sample Output

tokenized_input = tokenizer(
        text[0],
        padding=True,
        truncation=False,
        return_tensors='pt'
    )

sample_output = model(**tokenized_input)

In [20]:
print(tokenized_input)

{'input_ids': tensor([[     2,     26,  29571,  34458,   8101, 146537,   8132,    686,  10322,
         173051,      8,  16417,  42525,   2825,  97305,    335,      8,  38845,
          85415,      3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [21]:
print(sample_output)

BaseModelOutputWithPooling(last_hidden_state=tensor([[[-0.0132, -0.0055, -0.0209,  ..., -0.0215,  0.0071, -0.0089],
         [-0.7908, -0.2381, -0.1734,  ...,  0.0841, -0.1250,  0.3261],
         [ 0.3463, -0.2018, -0.1307,  ...,  0.3810,  0.1111, -0.0056],
         ...,
         [ 0.2304, -0.1409, -0.0564,  ..., -0.0202, -0.1202, -0.0020],
         [ 0.7018,  0.0072, -0.1355,  ...,  0.0876, -0.0934, -0.2410],
         [-0.0132, -0.0055, -0.0209,  ..., -0.0215,  0.0071, -0.0089]]],
       grad_fn=<NativeLayerNormBackward>), pooler_output=tensor([[-5.7640e-02,  4.8384e-02,  1.3850e-02, -5.5637e-03,  4.4495e-02,
          8.3772e-02,  3.8352e-02, -1.2101e-02, -1.3811e-02,  6.1028e-02,
         -9.2508e-03,  4.7674e-02,  1.9804e-02, -1.7189e-02,  5.1100e-02,
         -2.2018e-02, -4.3466e-02,  7.9840e-03, -1.1486e-01, -8.7218e-02,
          8.0016e-02,  7.8083e-02, -3.5701e-02, -9.7573e-02,  3.3622e-03,
         -3.2804e-02, -5.1533e-02, -5.4548e-03,  2.6833e-02,  8.7232e-03,
          2.

In [22]:
print(sample_output.pooler_output.cpu().detach().numpy().shape)

(1, 768)


In [23]:
# Tokenizing Input Data

input = []

for i in text:
    tokenized_input = tokenizer(
        i,
        padding=True,
        truncation=False,
        return_tensors='pt'
    )
    input.append(tokenized_input)

In [24]:
# Embedding the Input Data

output = []

j = 1

for i in range(len(input)):
    
    model_output = model(**input[i])
    model_output = model_output.pooler_output.cpu().detach().numpy()
    output.append(model_output)
    
    # Saving Embedded Input Data to Disk

    if ((i + 1) % 200) == 0:
        
        file_name = "./IndicBert-Embedded-TrainData/output" + str(j) + ".pickle"
        
        with open(file_name, "wb") as fp:   
            # Pickling
            pickle.dump(output, fp)
        
        print(file_name + " done")
        output = []
        j += 1
        
file_name = "./IndicBert-Embedded-TrainData/output" + str(j) + ".pickle"

with open(file_name, "wb") as fp:   
    # Pickling
    pickle.dump(output, fp)

print(file_name + " done")
output = []

./IndicBert-Embedded-TrainData/output1.pickle done
./IndicBert-Embedded-TrainData/output2.pickle done
./IndicBert-Embedded-TrainData/output3.pickle done
./IndicBert-Embedded-TrainData/output4.pickle done
./IndicBert-Embedded-TrainData/output5.pickle done
./IndicBert-Embedded-TrainData/output6.pickle done


In [25]:
# Loading Embedded Input Data from Disk

output = []

for i in range(6):
    file_name = "./IndicBert-Embedded-TrainData/output" + str(i + 1) + ".pickle"
    with open(file_name, "rb") as fp:   #Pickling
        file_output = pickle.load(fp)
        for x in file_output:
            output.append(x)
    print(file_name + " done")

X = output
output = []

./IndicBert-Embedded-TrainData/output1.pickle done
./IndicBert-Embedded-TrainData/output2.pickle done
./IndicBert-Embedded-TrainData/output3.pickle done
./IndicBert-Embedded-TrainData/output4.pickle done
./IndicBert-Embedded-TrainData/output5.pickle done
./IndicBert-Embedded-TrainData/output6.pickle done


In [26]:
X = np.array(X)
X.shape

(1000, 1, 768)

In [27]:
X = X.reshape(1000, 768)
X.shape

(1000, 768)

In [28]:
y = label
y.shape

(1000,)

## **Train Test Split**

In [29]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [30]:
X_train.shape

(800, 768)

In [31]:
X_val.shape

(200, 768)

## **Logistic Regression**

In [32]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_val)
print(classification_report(y_val, lr_pred))

              precision    recall  f1-score   support

           0       0.62      1.00      0.77       125
           1       0.00      0.00      0.00        75

    accuracy                           0.62       200
   macro avg       0.31      0.50      0.38       200
weighted avg       0.39      0.62      0.48       200



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## **Support Vector Machine**

In [33]:
svm = SVC()
svm.fit(X_train, y_train)
svm_pred = svm.predict(X_val)
print(classification_report(y_val, svm_pred))

              precision    recall  f1-score   support

           0       0.62      1.00      0.77       125
           1       0.00      0.00      0.00        75

    accuracy                           0.62       200
   macro avg       0.31      0.50      0.38       200
weighted avg       0.39      0.62      0.48       200



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## **Naive Bayes**

In [34]:
nb = GaussianNB()
nb.fit(X_train, y_train)
nb_pred = nb.predict(X_val)
print(classification_report(y_val, nb_pred))

              precision    recall  f1-score   support

           0       0.66      0.45      0.53       125
           1       0.40      0.61      0.48        75

    accuracy                           0.51       200
   macro avg       0.53      0.53      0.51       200
weighted avg       0.56      0.51      0.51       200



## **Stochastic Gradient Descent**

In [35]:
sgd = SGDClassifier()
sgd.fit(X_train, y_train)
sgd_pred = sgd.predict(X_val)
print(classification_report(y_val, sgd_pred))

              precision    recall  f1-score   support

           0       0.80      0.03      0.06       125
           1       0.38      0.99      0.55        75

    accuracy                           0.39       200
   macro avg       0.59      0.51      0.30       200
weighted avg       0.64      0.39      0.24       200



## **K Nearest Neighbours**

In [36]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_val)
print(classification_report(y_val, knn_pred))

              precision    recall  f1-score   support

           0       0.61      0.65      0.63       125
           1       0.34      0.31      0.32        75

    accuracy                           0.52       200
   macro avg       0.48      0.48      0.48       200
weighted avg       0.51      0.52      0.51       200



## **Decision Tree**

In [37]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
dt_pred = dt.predict(X_val)
print(classification_report(y_val, dt_pred))

              precision    recall  f1-score   support

           0       0.60      0.51      0.55       125
           1       0.35      0.44      0.39        75

    accuracy                           0.48       200
   macro avg       0.48      0.48      0.47       200
weighted avg       0.51      0.48      0.49       200



## **Random Forest**

In [38]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_val)
print(classification_report(y_val, rf_pred))

              precision    recall  f1-score   support

           0       0.63      0.81      0.71       125
           1       0.40      0.21      0.28        75

    accuracy                           0.58       200
   macro avg       0.52      0.51      0.49       200
weighted avg       0.54      0.58      0.55       200



## **Majority Voting**

In [40]:
mv_pred = []

for i in range(len(lr_pred)):
    one = 0
    zero = 0
    # Only having RF, KNN & DT in predictions 
    # due poor performance of other models
    predictions = [rf_pred[i], knn_pred[i], nb_pred[i]]
    for pred in predictions:
        if pred == 1: one += 1
        if pred == 0: zero +=1
    if one > zero: mv_pred.append(1)
    else: mv_pred.append(0)

mv_pred = np.array(mv_pred)

print(classification_report(y_val, mv_pred))

              precision    recall  f1-score   support

           0       0.61      0.66      0.64       125
           1       0.35      0.31      0.33        75

    accuracy                           0.53       200
   macro avg       0.48      0.49      0.48       200
weighted avg       0.52      0.53      0.52       200

